In [1]:
import pandas as pd
import numpy as np

In [2]:
dailydata = pd.read_csv('../data/co2-ppm-daily_csv.csv')

In [22]:
dates = [dailydata.iloc[i][0] for i in range(len(dailydata))]

In [25]:
_1959_dates = [i for i in dates if i.split('-')[0] == '1959']

In [28]:
dailydata.iloc[_1959_dates]

ValueError: invalid literal for int() with base 10: '1959-01-01'